In [32]:
import tensorflow as tf
import numpy as np
from datetime import datetime

In [12]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [13]:
X = tf.placeholder(tf.float32, shape = [None, n_inputs], name = "X")
y = tf.placeholder(tf.int64, shape = [None], name = "y")

In [14]:
def neuron_layer(X, n_neurons, name = "layer", activation = None) :
    with tf.name_scope(name) :
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs + n_neurons)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev = stddev)
        W = tf.Variable(init, name = "kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name = "bias")
        z = tf.matmul(X, W) + b
        if activation :
            return activation(z)
        else :
            return z

In [17]:
with tf.name_scope("dnn") :
    hidden1 = neuron_layer(X, n_hidden1, name = "hidden1", activation = tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name = "hidden2", activation = tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name = "outputs")
    # hidden1 = tf.layers.dense(X, n_hidden1, name = "hidden1", activation = tf.nn.relu)
    # hidden2 = tf.layers.dense(hidden1, n_hidden2, name = "hidden2", activation = tf.nn.relu)
    # logits = tf.layers.dense(hidden2, n_outputs, name = "outputs")

In [19]:
with tf.name_scope("loss") :
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels = y)
    loss = tf.reduce_mean(xentropy, name = "loss")

In [20]:
learning_rate = 0.01

In [22]:
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [23]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [24]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [25]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [38]:
n_epochs = 400
batch_size = 50
n_batches_per_epoch = mnist.train.num_examples // batch_size

In [39]:
now = datetime.utcnow().strftime("%Y%M%d%H%M%s")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [40]:
mse_summary = tf.summary.scalar("MSE", loss)
summary_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [ ]:
with tf.Session() as sess :
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(n_batches_per_epoch):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict = {X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict = {X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict = {X: mnist.test.images, y: mnist.test.labels})
        print(epoch, "Train accuracy :", acc_train, "Test accuracy :", acc_test)
        
        summary_str = mse_summary.eval(feed_dict = {X: X_batch, y: y_batch})
        summary_writer.add_summary(summary_str, epoch)
    
    save_path = saver.save(sess, "my_model_final_checkpoint.ckpt")

0 Train accuracy : 0.94 Test accuracy : 0.912
1 Train accuracy : 0.94 Test accuracy : 0.9253
2 Train accuracy : 0.9 Test accuracy : 0.9347
3 Train accuracy : 0.94 Test accuracy : 0.94
4 Train accuracy : 0.98 Test accuracy : 0.9456
11 Train accuracy : 0.94 Test accuracy : 0.9645
12 Train accuracy : 1.0 Test accuracy : 0.9653
13 Train accuracy : 1.0 Test accuracy : 0.9671
14 Train accuracy : 1.0 Test accuracy : 0.9682
15 Train accuracy : 0.98 Test accuracy : 0.9682
16 Train accuracy : 1.0 Test accuracy : 0.9698
17 Train accuracy : 1.0 Test accuracy : 0.9699
18 Train accuracy : 1.0 Test accuracy : 0.9698
19 Train accuracy : 1.0 Test accuracy : 0.9709
20 Train accuracy : 1.0 Test accuracy : 0.9723
21 Train accuracy : 1.0 Test accuracy : 0.973
22 Train accuracy : 1.0 Test accuracy : 0.9726
23 Train accuracy : 0.98 Test accuracy : 0.972


In [31]:
print(save_path)

my_model_final_checkpoint.ckpt


In [ ]:
summary_writer.close()